\Neural Network,
Bagging - RF,
Boosting - AdaBoost,
Gradient Boosting

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

### For all the well-being parameters

In [ ]:
#important features
feature_names_alert = ['filteredDemographicVO2Max','nremhr','bmi','minutesAsleep','min_goal','minutes_below_default_zone_1','max_goal','resting_hr','full_sleep_breathing_rate','rmssd','sleep_efficiency','sleep_light_ratio','very_active_minutes','minutes_in_default_zone_2','sleep_wake_ratio','nightly_temperature','sleep_duration','age','sleep_points_percentage']
feature_names_happy = ['filteredDemographicVO2Max', 'sleep_points_percentage', 'sleep_wake_ratio', 'stress_score', 'exertion_points_percentage', 'sleep_rem_ratio', 'nremhr', 'rmssd', 'min_goal', 'nightly_temperature', 'sleep_efficiency', 'steps', 'minutesAfterWakeup', 'daily_temperature_variation', 'gender_FEMALE', 'full_sleep_breathing_rate', 'sleep_light_ratio']
feature_names_rested = ['filteredDemographicVO2Max', 'daily_temperature_variation', 'exertion_points_percentage', 'rmssd', 'bmi', 'lightly_active_minutes', 'nremhr', 'sleep_rem_ratio', 'sleep_deep_ratio', 'sleep_efficiency', 'nightly_temperature', 'gender_FEMALE', 'min_goal', 'sleep_wake_ratio', 'minutesAfterWakeup', 'full_sleep_breathing_rate', 'max_goal', 'sleep_points_percentage', 'minutesAwake', 'responsiveness_points_percentage', 'gender_MALE', 'resting_hr', 'distance', 'sedentary_minutes']
feature_names_sad = ['resting_hr','full_sleep_breathing_rate', 'filteredDemographicVO2Max', 'max_goal']
feature_names_tensed = ['sleep_light_ratio', 'min_goal', 'rmssd', 'bmi', 'filteredDemographicVO2Max', 'max_goal', 'nremhr', 'steps', 'gender_FEMALE', 'stress_score']
feature_names_tired = ['sleep_light_ratio', 'minutesAfterWakeup', 'sleep_rem_ratio', 'lightly_active_minutes', 'sleep_wake_ratio', 'sleep_deep_ratio', 'filteredDemographicVO2Max', 'gender_MALE', 'sleep_points_percentage', 'daily_temperature_variation', 'responsiveness_points_percentage', 'min_goal', 'stress_score', 'minutesToFallAsleep', 'bpm', 'gender_FEMALE', 'resting_hr', 'minutesAwake', 'bmi', 'full_sleep_breathing_rate']
feature_names_entertainment = ['very_active_minutes', 'sleep_wake_ratio', 'sleep_duration', 'sleep_rem_ratio', 'sleep_efficiency', 'distance', 'nremhr', 'minutesAwake']

feature_used_list = [feature_names_alert, feature_names_happy, feature_names_rested, feature_names_sad, feature_names_tensed, feature_names_tired, feature_names_entertainment]

In [ ]:
X_list = ['ALERT','HAPPY','RESTED/RELAXED','SAD','TENSE/ANXIOUS','TIRED','ENTERTAINMENT']

### NN Best parameter tuning

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.preprocessing import StandardScaler

file_path1 = '/content/cleaned_data.csv'
file_path2 = '/content/cleaned_data_only_input.csv'
data1 = pd.read_csv(file_path1)
data2 = pd.read_csv(file_path2)
best_params_list = []
for target_col in X_list:
  X = data2.values
  y = data1[target_col].values
  scaler = StandardScaler()
  X = scaler.fit_transform(X)
  param_grid = {
      'hidden_layer_sizes': [ (128,64), (64, 32), (64, 64, 16), (128,64,32)],
      'activation': ['relu', 'tanh'],
      'alpha': [0.0001, 0.001, 0.01],
      'learning_rate_init': [0.0001, 0.001, 0.01, 0.1],
      'max_iter': [50],
      'random_state': [1, 42, 100]
  }

  grid_search = GridSearchCV(MLPClassifier(), param_grid, cv=3)
  grid_search.fit(X, y)
  best_params = grid_search.best_params_
  best_params_list.append(best_params)

In [ ]:
print(best_params_list)

### Stacking Classifier

In [ ]:
predictions_df = pd.DataFrame()
for feat, op_data, best_params in zip(feature_used_list, X_list,best_params_list):
  base_models = [
    ('Random Forest', RandomForestClassifier(n_estimators=50, random_state=42)),
    ('Gradient Boosting', GradientBoostingClassifier(n_estimators=50, random_state=42)),
    ('Neural Networks', MLPClassifier(**best_params)),
    ('AdaBoost', AdaBoostClassifier(n_estimators=50, random_state=42)),
    ]
  X = data2[feat]
  y = data1[op_data]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  print("Predicting for:", op_data)
  for name, model in base_models:
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    y_pred_test = model.predict(X_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print(f'{name} Train Accuracy: {accuracy_train:.4f}, Test Accuracy: {accuracy_test:.4f}')

# Defining Stacking model
  stacking_model = StackingClassifier(estimators=base_models[:-1], final_estimator=base_models[-1][1])
  stacking_model.fit(X_train, y_train)
  y_pred_stacking_train = stacking_model.predict(X_train)
  accuracy_stacking_train = accuracy_score(y_train, y_pred_stacking_train)

  y_pred_stacking_test = stacking_model.predict(X_test)
  accuracy_stacking_test = accuracy_score(y_test, y_pred_stacking_test)
  print(f'Stacking Train Accuracy: {accuracy_stacking_train:.4f}, Test Accuracy: {accuracy_stacking_test:.4f}')
  predictions_df[op_data] = y_pred_stacking_test

predictions_df.to_csv('predictions.csv', index=False)